# Language Models 1 | Inference (Ollama)

**For this notebook to run, you need an ollama server running (`ollama serve` in a terminal)!**

[ollama.com](https://ollama.com)  
[ollama Github doc](https://github.com/ollama/ollama)  
[ollama Python doc](https://github.com/ollama/ollama-python)  
[markdown doc](https://python-markdown.github.io/reference/)

Here we will be using the library [Ollama](https://ollama.com) ([github](https://github.com/ollama/ollama) – itself based on a lower-level project called [llama.cpp](https://github.com/ggml-org/llama.cpp)), to run LLMs. locally. The other place I recommend looking up if you want to know more is [Huggingface](https://huggingface.co/), which is a set of libraries, a hub to share models and datasets, and a provider of many tutorials.

In [ ]:
import ollama
import IPython
import numpy as np

import markdown
import strip_markdown

import base64
from pathlib import Path

In [ ]:
# util function to check if a model is available: if not: download it
def check_model_and_pull(m_name):
    # test if the model is downloaded, if not pull (download) from the server
    if m_name not in [m.model for m in ollama.list().models]:
        print(f"model '{m_name}' not found, downloading...")
        # pull/download model
        ollama.pull(m_name)
    else:
        print(f"model: `{m_name}` found!")

[Gemma 3 family](https://ollama.com/library/gemma3)  
[Gemma 3 270m](https://ollama.com/library/gemma3:270m)

In [ ]:
model_name = "gemma3:270m"

In [ ]:
# test if the model is downloaded, if not pull from the server
check_model_and_pull(model_name)

response = ollama.chat(
    model=model_name, 
    messages=[
      { "role": "user", "content": "Why is the sky blue?" },
    ]
  )

In [ ]:
print(response)

In [ ]:
print(response["message"]["content"])
# you can also access fields directly from the response object
# print(response.message.content)

## Options

Full list [here](https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-chat-completion).

In [ ]:
print(
    ollama.chat(
        model=model_name,
        messages=[{ "role": "user", "content": "Why is the sky blue?" },],
        options={
            "temperature": 0.,
        })["message"]["content"]
)

In [ ]:
print(
    ollama.chat(
        model=model_name,
        messages=[{ "role": "user", "content": "Why is the sky blue?" },],
        options={
            "temperature": 10.,
        })["message"]["content"]
)

In [ ]:
print(
    ollama.chat(
        model=model_name,
        messages=[{ "role": "user", "content": "Why is the sky blue?" },],
        options={
            # stop after a specific number of tokens
            "num_predict": 5
        })["message"]["content"]
)

In [ ]:
print(
    ollama.chat(
        model=model_name,
        messages=[{ "role": "user", "content": "Why is the sky blue?" },],
        options={
            # use a special stopping sequence
            "stop": ["a"],
        })["message"]["content"]
)

## Note: handle markdown

[Markdown](https://pypi.org/project/Markdown/)  
[strip-markdown](https://pypi.org/project/strip-markdown/)

In [ ]:
response.message.content

In [ ]:
def md2html(text):
    return markdown.markdown(text)

def print_html(raw_html):
    IPython.display.display_html(raw_html, raw=True)

print_html(md2html(response.message.content))

In [ ]:
def strip_md(text):
    return strip_markdown.strip_markdown(text)

print(strip_md(response.message.content))

## Gradual printing / streaming responses

In [ ]:
stream = ollama.chat(
    model=model_name,
    messages=[{"role": "user", "content": "Why is the sky blue?"}],
    stream=True,
)

for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

In [ ]:
stream = ollama.chat(
    model=model_name,
    messages=[{"role": "user", "content": "Why is the sky blue?"}],
    stream=True,
)

text = ""
for chunk in stream:
    text += chunk["message"]["content"]
    IPython.display.clear_output(wait=True)
    print_html(md2html(text))

### Extra: Generate experiment: removing the template!

[doc](https://github.com/ollama/ollama-python?tab=readme-ov-file#generate), [REST API](https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion)

In [ ]:
prompt = "Hi, my name is Jeremie."

print(
    ollama.generate(
        model=model_name,
        prompt=prompt,
        # if `True`` no formatting will be applied to the prompt!
        raw=True,
        options= {
            # here we limit the output to 50 tokens only
            "num_predict": 50,
            # you can play with the temperature if you want
            # "temperature": .9,
            }
        )["response"]
)

Looking at the [template](https://ollama.com/library/gemma3:270m/blobs/4b19ac7dd2fb) (also [test it live in Tiktokenizer!](https://tiktokenizer.vercel.app/?model=google%2Fgemma-7b), an earlier model in this series but the template looks the same), I can manually recreate the text that the model actually reads, so that it behaves again like a chatbot:

In [ ]:
# the structure has these special markers + new lines
prompt = """<start_of_turn>user
Hi, my name is Jeremie.<end_of_turn>
<start_of_turn>model
"""

# # if you use this, the model will actually start answering as the *user*!
# prompt = """<start_of_turn>user
# Hi, my name is Jeremie.<end_of_turn>
# <start_of_turn>model
# Hi, I'm a weird machine, how can I be of help?<end_of_turn>
# <start_of_turn>user
# """

print(
    ollama.generate(
        model=model_name,
        prompt= prompt,
        # if `True`` no formatting will be applied to the prompt!
        raw=True,
        options= {
            # here we limit the output to 50 tokens only
            "num_predict": 50,
            # "temperature": .9,
            
            }
        )["response"]
)

## Extra: multimodality

[Gemma 3, 4b doc](https://ollama.com/library/gemma3:4b)

**Multimodality** refers to the fact that models are trained, and therefore can understand/interact with, multiple types of data: in the most common case, it's *text* and *images* (could also be sound, video, sensory data from a robot, etc.). Here you can see an example of a model that is able to read/understand images.

Adapted from the [gemma3 example](https://github.com/ollama/ollama-python/blob/main/examples/multimodal-chat.py) – a bigger model, taking around 2-3GB in RAM. (See also the [llava example](https://github.com/ollama/ollama-python/blob/main/examples/multimodal-generate.py)).

In [ ]:
# our new model name 
model_name = "gemma3:4b"

# download it if not present
check_model_and_pull(model_name)

[base64 doc](https://docs.python.org/3/library/base64.html)  
[w<sup>3</sup> tutorial](https://www.w3schools.com/Python/ref_module_base64.asp)  
[base64 RealPython tutorial](https://realpython.com/python-serialize-data/)

Ollama supports feeding the image as a path, a `base64` string (a kind of encoding, using only ASCII characters), or raw bytes.

In [ ]:
file_option = 0

# pass in the path to the image
path_or_img = "images/spock.jpg"

if file_option == 1:
    # you can also pass in base64 encoded image data
    path_or_img = base64.b64encode(Path(path_or_img).read_bytes()).decode()
elif file_option == 2:
    # or the raw bytes
    path_or_img = Path(path_or_img).read_bytes()

response = ollama.chat(
    # note that 
    model=model_name,
    messages=[
        {
            'role': 'user',
            'content': 'What is in this image? Be detailed.',
            # we just add an 'image' key:value pair to our message object
            # the value is a list, containing one or more images (as paths/base64 str/bytes)
            'images': [path_or_img],
        }
    ],
)

print_html(md2html(response.message.content))

### Extra: wanna see the `base64` string?

In [ ]:
# read raw bytes, then encode as b64 (still bytes), then decode as string
image_as_str = base64.b64encode(Path(path_or_img).read_bytes()).decode()
# only the first 100 characters
image_as_str[:100]

## Extra: tools, web browsing

Recent models have even more functionalities, such as:
- using tools/calling functions ([tools example](https://github.com/ollama/ollama-python/blob/main/examples/tools.py), [multi-tool example](https://github.com/ollama/ollama-python/blob/main/examples/multi-tool.py), [async tools](https://github.com/ollama/ollama-python/blob/main/examples/async-tools.py))
- 'thinking' (namely generate more tokens to arrive at an answer) ([thinking chat example](https://github.com/ollama/ollama-python/blob/main/examples/thinking.py), [thinking generate example](https://github.com/ollama/ollama-python/blob/main/examples/thinking-generate.py), [thinking levels examples](https://github.com/ollama/ollama-python/blob/main/examples/thinking-levels.py))
- web-browsing ([qwen example](https://github.com/ollama/ollama-python/blob/main/examples/web-search.py), [gpt-oss example](https://github.com/ollama/ollama-python/blob/main/examples/web-search-gpt-oss.py))


Beware, many recent models require a lot of memory. For example **gpt-oss-20b** requires just under 12GB of RAM...